# Fix pathing

In [1]:
import sys


sys.path.append("../..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))))


# Imports

In [3]:
import plotter
import datahandler

import matplotlib.pyplot
import numpy as np
import pandas as pd
import seaborn as sns
import IPython.display
import json
import matplotlib.pyplot as plt


# Constants

In [12]:
FOLDER_NAMES = ["ex_7_prediction_2018_16_NSGA2", "ex_7_prediction_2019_16_NSGA2"]
FILE_INFO = {
    "ex_7_prediction_2018_16_NSGA2": (2018.16),
    "ex_7_prediction_2019_16_NSGA2": (2019.16),
}

FOLDER_PATH = os.path.join(os.path.dirname(constants.PROJECT_DIRECTORY_PATH), "Simulator", "data")

SEEDS = list(range(10))

deep_palette = sns.color_palette("deep")
deep_colors = {
    "blue": deep_palette[0],
    "orange": deep_palette[1],
    "green": deep_palette[2],
    "red": deep_palette[3],
}

bright_palette = sns.color_palette("bright")
bright_colors = {
    "blue": bright_palette[0],
    "orange": bright_palette[1],
    "green": bright_palette[2],
    "red": bright_palette[3],
}


In [5]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(False, False, True)


Loading dataset:   0%|          | 0/2 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


# Methods

In [6]:
def load_json():
    dataset = {}

    for folder_name in FOLDER_NAMES:
        for seed in SEEDS:
            simulated_year = FILE_INFO[folder_name]

            with open(os.path.join(FOLDER_PATH, folder_name, f"heuristic_seed={seed}.json"), 'r') as file:
                dataset[(simulated_year, seed)] = json.load(file)

    return dataset


In [7]:
def add_info(dataframe: pd.DataFrame) -> pd.DataFrame:
    historic_steps = {
        "duration_incident_creation": ("time_call_received", "time_incident_created"),
        "duration_resource_appointment": ("time_incident_created", "time_resource_appointed"),
        "duration_resource_preparing_departure": ("time_resource_appointed", "time_ambulance_dispatch_to_scene"),
        "duration_dispatching_to_scene": ("time_ambulance_dispatch_to_scene", "time_ambulance_arrived_at_scene"),
        "duration_at_scene": ("time_ambulance_arrived_at_scene", "time_ambulance_dispatch_to_hospital", "time_ambulance_available"),
        "duration_dispatching_to_hospital": ("time_ambulance_dispatch_to_hospital", "time_ambulance_arrived_at_hospital"),
        "duration_at_hospital": ("time_ambulance_arrived_at_hospital", "time_ambulance_available")
    }

    for step, times in historic_steps.items():
        if len(times) == 3:
            dataframe.loc[dataframe[times[1]].isna(), step] = (dataframe[times[2]] - dataframe[times[0]]).dt.total_seconds()
            dataframe.loc[~dataframe[times[1]].isna(), step] = (dataframe[times[1]] - dataframe[times[0]]).dt.total_seconds()
        else:
            dataframe[step] = (dataframe[times[1]] - dataframe[times[0]]).dt.total_seconds()
    
    response_time_cols = [
        'duration_incident_creation',
        'duration_resource_appointment',
        'duration_resource_preparing_departure',
        'duration_dispatching_to_scene'
    ]
    dataframe['total_response_time'] = dataframe[response_time_cols].sum(axis=1)

    dataframe["complied"] = np.nan
    # Define the criteria for response times
    criteria = {
        ('A', True): 12 * 60,
        ('A', False): 25 * 60,
        ('H', True): 30 * 60,
        ('H', False): 40 * 60
    }

    for (triage, urban), group in dataframe.groupby(['triage_impression_during_call', 'urban_settlement_ssb']):
        limit = criteria.get((triage, urban))
        if limit is not None:
            dataframe.loc[group.index, 'complied'] = group['total_response_time'] < limit

    return dataframe


In [8]:
def compute_metrics(dataset):
    dataframe = pd.DataFrame()

    dataframe["simulated_year"] = 0
    dataframe["seed"] = 0
    dataframe["generation"] = 0
    dataframe["individual"] = 0
    dataframe["diversity"] = 0
    dataframe["front_number"] = np.nan
    dataframe["crowding_distance"] = np.nan
    dataframe["percentage_violations"] = 0
    dataframe["percentage_violations_urban"] = 0
    dataframe["percentage_violations_rural"] = 0
    dataframe["avg_response_time_urban_a"] = 0
    dataframe["avg_response_time_urban_h"] = 0
    dataframe["avg_response_time_urban_v1"] = 0
    dataframe["avg_response_time_rural_a"] = 0
    dataframe["avg_response_time_rural_h"] = 0
    dataframe["avg_response_time_rural_v1"] = 0

    for (simulated_year, seed), data in dataset.items():
        generation = list(range(len(data['diversity'])))[-1]

        individuals = list(range(len(data['percentage_violations'][generation])))

        diversity = data['diversity'][generation][0]

        for individual in individuals:
            objectives = []
            objective_keys = [
                "front_number",
                "crowding_distance",
                "percentage_violations",
                "percentage_violations_urban",
                "percentage_violations_rural",
                "avg_response_time_urban_a",
                "avg_response_time_urban_h",
                "avg_response_time_urban_v1",
                "avg_response_time_rural_a",
                "avg_response_time_rural_h",
                "avg_response_time_rural_v1",
            ]

            for objective_key in objective_keys:
                if objective_key in data:
                    objective = data[objective_key][generation][individual]
                else:
                    objective = np.nan
                
                objectives.append(objective)

            # add row
            dataframe.loc[len(dataframe.index)] = [
                simulated_year,
                seed,
                generation,
                individual,
                diversity,
                *objectives
            ]
    
    dataframe.drop_duplicates(
        subset=[
            "simulated_year",
            "seed",
            "generation",
            "diversity",
            "front_number",
            "percentage_violations",
            "percentage_violations_urban",
            "percentage_violations_rural",
            "avg_response_time_urban_a",
            "avg_response_time_urban_h",
            "avg_response_time_urban_v1",
            "avg_response_time_rural_a",
            "avg_response_time_rural_h",
            "avg_response_time_rural_v1",
        ],
        inplace=True
    )
    
    return dataframe


In [9]:
def load_csv(filename = "events"):
    dataframes = []
    for seed in list(range(10)):
        dataframes.append(pd.read_csv(os.path.join(FOLDER_PATH, filename + "_seed=" + str(seed) + ".csv")))
    
    dataframe = pd.concat(dataframes, ignore_index=True)

    response_time_cols = [
        'duration_incident_creation',
        'duration_resource_appointment',
        'duration_resource_preparing_departure',
        'duration_dispatching_to_scene'
    ]
    dataframe['total_response_time'] = dataframe[response_time_cols].sum(axis=1)

    dataframe["complied"] = np.nan
    # Define the criteria for response times
    criteria = {
        ('A', True): 12 * 60,
        ('A', False): 25 * 60,
        ('H', True): 30 * 60,
        ('H', False): 40 * 60
    }

    for (triage, urban), group in dataframe.groupby(['triage_impression_during_call', 'urban']):
        if triage == 'V1':
            continue
    
        limit = criteria.get((triage, urban))
        if limit is not None:
            dataframe.loc[group.index, 'complied'] = group['total_response_time'] < limit

    return dataframe


In [10]:
def print_info(dataframe: pd.DataFrame):
    filtered_df = dataframe.dropna(subset=['complied'])

    # compliance
    numerator = filtered_df['complied'].count()
    true_count = filtered_df['complied'].sum()

    compliance_rate = true_count / numerator

    print(f"Compliance Rate: {(abs(compliance_rate - 1) * 100):.2f}")

    # complaince (U)
    # compliance (R)
    compliance_rates = filtered_df.groupby('urban')['complied'].agg(lambda x: x.mean())

    groups = []
    rates = []

    for group, rate in compliance_rates.items():
        groups.append(group)
        rates.append(rate)
    
    for group, rate in zip(groups[::-1], rates[::-1]):
        print(f"Compliance Rate for {'Urban' if group else 'Non-Urban'}: {(abs(rate - 1) * 100):.2f}")


# Main

In [13]:
dataset = load_json()
dataframe = compute_metrics(dataset)

dataframe.info()


<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 0 to 380
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   simulated_year               26 non-null     float64
 1   seed                         26 non-null     float64
 2   generation                   26 non-null     float64
 3   individual                   26 non-null     float64
 4   diversity                    26 non-null     float64
 5   front_number                 26 non-null     float64
 6   crowding_distance            26 non-null     float64
 7   percentage_violations        26 non-null     float64
 8   percentage_violations_urban  26 non-null     float64
 9   percentage_violations_rural  26 non-null     float64
 10  avg_response_time_urban_a    26 non-null     float64
 11  avg_response_time_urban_h    26 non-null     float64
 12  avg_response_time_urban_v1   26 non-null     float64
 13  avg_response_time_rural_a 

In [14]:
simulated_year = 2018.16


In [15]:
indices = []

seed = 0
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 1
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 2
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 3
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 4
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 5
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 6
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 7
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 8
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

seed = 9
filtered_df = dataframe[
    (dataframe["simulated_year"] == simulated_year) & 
    (dataframe["seed"] == seed) & 
    (dataframe["generation"] == dataframe[(dataframe["simulated_year"] == simulated_year) & (dataframe["seed"] == seed)]["generation"].max()) &
    (dataframe["front_number"] == 0)
]
indices.append(filtered_df.index[0])
IPython.display.display(filtered_df)

print(indices)


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
0,2018.16,0.0,147.0,0.0,0.05,0.0,1.797690e+308,0.144444,0.159509,0.0,651.031,1263.98,3687.8,890.364,1483.67,3913.25


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
20,2018.16,1.0,55.0,0.0,0.05,0.0,1.797690e+308,0.138889,0.153374,0.0,663.734,1253.44,3598.96,718.818,1193.83,3585.5


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
40,2018.16,2.0,230.0,0.0,0.05,0.0,1.797690e+308,0.144444,0.159509,0.0,651.672,1301.77,3483.94,849.364,1362.83,5277.5


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
60,2018.16,3.0,335.0,0.0,0.05,0.0,1.797690e+308,0.133333,0.147239,0.0,639.25,1240.19,3301.0,810.273,1120.33,2994.5


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
80,2018.16,4.0,383.0,0.0,0.1,0.0,1.797690e+308,0.127778,0.134969,0.058824,622.891,1261.03,3623.81,987.273,1099.5,4621.75
82,2018.16,4.0,383.0,2.0,0.1,0.0,1.797690e+308,0.155556,0.171779,0.000000,650.219,1289.87,4067.68,802.727,1239.5,4051.00


...

,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
120,2018.16,6.0,16.0,0.0,0.05,0.0,1.797690e+308,0.177778,0.196319,0.0,659.859,1308.73,3709.14,718.909,1088.17,5566.75


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
140,2018.16,7.0,118.0,0.0,0.05,0.0,1.797690e+308,0.155556,0.171779,0.0,619.859,1244.64,3687.37,881.727,1272.33,4830.0


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
160,2018.16,8.0,397.0,0.0,0.15,0.0,1.797690e+308,0.155556,0.165644,0.058824,657.969,1219.39,3573.90,937.273,1203.83,4008.75
161,2018.16,8.0,397.0,1.0,0.15,0.0,1.797690e+308,0.155556,0.165644,0.058824,657.719,1244.55,3600.87,919.000,1333.33,4008.75
162,2018.16,8.0,397.0,2.0,0.15,0.0,1.797690e+308,0.177778,0.196319,0.000000,652.453,1353.35,3683.48,692.636,1198.00,3132.00


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
180,2018.16,9.0,189.0,0.0,0.05,0.0,1.797690e+308,0.15,0.165644,0.0,629.703,1269.33,3726.24,942.818,1230.83,2650.75


[0, 20, 40, 60, 80, 100, 120, 140, 160, 180]


In [17]:
best_individuals_indices_2018_16 = [0, 20, 40, 60, 80, 100, 120, 140, 160, 180]
best_individuals_indices_2019_16 = [200, 220, 240, 260, 280, 300, 320, 340, 360, 380]


In [20]:
# DataFrame of best individuals from each seed
best_individuals_df = dataframe.loc[best_individuals_indices_2018_16]

IPython.display.display(best_individuals_df)

# Find the overall best individual across all seeds
overall_best_individual = best_individuals_df.loc[60]
overall_best_seed = overall_best_individual["seed"]
overall_best_fitness = overall_best_individual["percentage_violations"]
overall_best_vio_urban = overall_best_individual["percentage_violations_urban"]
overall_best_vio_rural = overall_best_individual["percentage_violations_rural"]
overall_best_individual_generations = overall_best_individual["generation"]

# Total amount of generations: count all unique generation numbers across all seeds
total_generations = filtered_df["generation"].nunique()
std_dev_fitness = best_individuals_df["percentage_violations"].std()

# Average fitness and average number of generations per seed
average_fitness_per_seed = best_individuals_df["percentage_violations"].mean()
average_generations_per_seed = best_individuals_df["generation"].mean()

print(f"Overall Best: Seed: {overall_best_seed}")
print(f"Overall Best: Fitness: {overall_best_fitness * 100:.2f}%")
print(f"Overall Best: Vio Urban: {overall_best_vio_urban * 100:.2f}%")
print(f"Overall Best: Vio Rural: {overall_best_vio_rural * 100:.2f}%")
print(f"Overall Best: Total Generations: {total_generations}")
print(f"Average Fitness per Seed: {average_fitness_per_seed * 100:.2f}%")
print(f"Average Generations per Seed: {average_generations_per_seed}")
print(f"Standard Deviation of Fitness: {std_dev_fitness * 100:.2f}%")


,simulated_year,seed,generation,individual,diversity,front_number,crowding_distance,percentage_violations,percentage_violations_urban,percentage_violations_rural,avg_response_time_urban_a,avg_response_time_urban_h,avg_response_time_urban_v1,avg_response_time_rural_a,avg_response_time_rural_h,avg_response_time_rural_v1
0,2018.16,0.0,147.0,0.0,0.05,0.0,1.797690e+308,0.144444,0.159509,0.000000,651.031,1263.98,3687.80,890.364,1483.67,3913.25
20,2018.16,1.0,55.0,0.0,0.05,0.0,1.797690e+308,0.138889,0.153374,0.000000,663.734,1253.44,3598.96,718.818,1193.83,3585.50
40,2018.16,2.0,230.0,0.0,0.05,0.0,1.797690e+308,0.144444,0.159509,0.000000,651.672,1301.77,3483.94,849.364,1362.83,5277.50
60,2018.16,3.0,335.0,0.0,0.05,0.0,1.797690e+308,0.133333,0.147239,0.000000,639.250,1240.19,3301.00,810.273,1120.33,2994.50
80,2018.16,4.0,383.0,0.0,0.10,0.0,1.797690e+308,0.127778,0.134969,0.058824,622.891,1261.03,3623.81,987.273,1099.50,4621.75
100,2018.16,5.0,112.0,0.0,0.05,0.0,1.797690e+308,0.144444,0.159509,0.000000,673.422,1255.55,3742.26,825.455,1221.00,3622.75
120,2018.16,6.0,16.0,0.0,0.05,0.0,1.797690e+308,0.177778,0.196319,0.000000,659.859,1308.73,3709.14,718.909,1088.17,5566.75
140,2018.16,7.0,118.0,0.0,0.05,0.0,1.797690e+308,0.155556,0.171779,0.000000,619.859,1244.64,3687.37,881.727,1272.33,4830.00
160,2018.16,8.0,397.0,0.0,0.15,0.0,1.797690e+308,0.155556,0.165644,0.058824,657.969,1219.39,3573.90,937.273,1203.83,4008.75
180,2018.16,9.0,189.0,0.0,0.05,0.0,1.797690e+308,0.150000,0.165644,0.000000,629.703,1269.33,3726.24,942.818,1230.83,2650.75


Overall Best: Seed: 3.0
Overall Best: Fitness: 13.33%
Overall Best: Vio Urban: 14.72%
Overall Best: Vio Rural: 0.00%
Overall Best: Total Generations: 1
Average Fitness per Seed: 14.72%
Average Generations per Seed: 198.2
Standard Deviation of Fitness: 1.39%


In [16]:
dataframe_historic = data_preprocessor.load_enhanced_incidents_dataframe()
dataframe_historic = add_info(dataframe_historic)


C:\Users\sindr\AppData\Local\Temp\ipykernel_6748\1518890614.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ True  True  True ...  True  True  True]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dataframe.loc[group.index, 'complied'] = group['total_response_time'] < limit


In [17]:
start_time = pd.to_datetime('07:00:00').time()
end_time = pd.to_datetime('18:59:59').time()

# filter rows where time is within the defined bounds
dataframe_historic = dataframe_historic[dataframe_historic['time_call_received'].dt.time.between(start_time, end_time)]

dataframe_historic["year"] = dataframe_historic["time_call_received"].dt.year
dataframe_historic["month"] = dataframe_historic["time_call_received"].dt.month
dataframe_historic["day"] = dataframe_historic["time_call_received"].dt.day



In [18]:
condition = ((dataframe_historic["year"] == 2018) & (dataframe_historic["month"] == 1) & (dataframe_historic["day"].isin([16])))
df = dataframe_historic[condition]

dataframe_all = df[df["triage_impression_during_call"] != "V1"]

dataframe_urban = df[(df["triage_impression_during_call"] != "V1") & (df["urban_settlement_ssb"] == True)]
dataframe_urban_acute = dataframe_urban[dataframe_urban["triage_impression_during_call"] == "A"]
dataframe_urban_urgent = dataframe_urban[dataframe_urban["triage_impression_during_call"] == "H"]

dataframe_rural = df[(df["triage_impression_during_call"] != "V1") & (df["urban_settlement_ssb"] == False)]
dataframe_rural_acute = dataframe_rural[dataframe_rural["triage_impression_during_call"] == "A"]
dataframe_rural_urgent = dataframe_rural[dataframe_rural["triage_impression_during_call"] == "H"]

print(f"Urban incidents below acute threshold: {abs((dataframe_urban_acute['complied'].sum() / dataframe_urban_acute.shape[0]) - 1) * 100:.2f}%")
print(f"Urban incidents below urgent threshold: {abs((dataframe_urban_urgent['complied'].sum() / dataframe_urban_urgent.shape[0]) - 1) * 100:.2f}%")
print()
print(f"Non-urban incidents below acute threshold: {abs((dataframe_rural_acute['complied'].sum() / dataframe_rural_acute.shape[0]) - 1) * 100:.2f}%")
print(f"Non-urban incidents below urgent threshold: {abs((dataframe_rural_urgent['complied'].sum() / dataframe_rural_urgent.shape[0]) - 1) * 100:.2f}%")
print()
print(f"Incidents below thresholds: {abs((dataframe_all['complied'].sum() / dataframe_all.shape[0]) - 1) * 100:.2f}%")


Urban incidents below acute threshold: 34.67%
Urban incidents below urgent threshold: 28.77%

Non-urban incidents below acute threshold: 14.29%
Non-urban incidents below urgent threshold: 0.00%

Incidents below thresholds: 30.77%


In [19]:
condition = ((dataframe_historic["year"] == 2019) & (dataframe_historic["month"] == 1) & (dataframe_historic["day"].isin([16])))
df = dataframe_historic[condition]

dataframe_all = df[df["triage_impression_during_call"] != "V1"]

dataframe_urban = df[(df["triage_impression_during_call"] != "V1") & (df["urban_settlement_ssb"] == True)]
dataframe_urban_acute = dataframe_urban[dataframe_urban["triage_impression_during_call"] == "A"]
dataframe_urban_urgent = dataframe_urban[dataframe_urban["triage_impression_during_call"] == "H"]

dataframe_rural = df[(df["triage_impression_during_call"] != "V1") & (df["urban_settlement_ssb"] == False)]
dataframe_rural_acute = dataframe_rural[dataframe_rural["triage_impression_during_call"] == "A"]
dataframe_rural_urgent = dataframe_rural[dataframe_rural["triage_impression_during_call"] == "H"]

print(f"Urban incidents below acute threshold: {abs((dataframe_urban_acute['complied'].sum() / dataframe_urban_acute.shape[0]) - 1) * 100:.2f}%")
print(f"Urban incidents below urgent threshold: {abs((dataframe_urban_urgent['complied'].sum() / dataframe_urban_urgent.shape[0]) - 1) * 100:.2f}%")
print()
print(f"Non-urban incidents below acute threshold: {abs((dataframe_rural_acute['complied'].sum() / dataframe_rural_acute.shape[0]) - 1) * 100:.2f}%")
print(f"Non-urban incidents below urgent threshold: {abs((dataframe_rural_urgent['complied'].sum() / dataframe_rural_urgent.shape[0]) - 1) * 100:.2f}%")
print()
print(f"Urban incidents below thresholds: {abs((dataframe_urban['complied'].sum() / dataframe_urban.shape[0]) - 1) * 100:.2f}%")
print(f"Rural incidents below thresholds: {abs((dataframe_rural['complied'].sum() / dataframe_rural.shape[0]) - 1) * 100:.2f}%")
print(f"Incidents below thresholds: {abs((dataframe_all['complied'].sum() / dataframe_all.shape[0]) - 1) * 100:.2f}%")


ZeroDivisionError: division by zero